In [1]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String
class US_Bill(Base):
    __tablename__ = 'us_bills'
    bill_num = Column(String, primary_key=True)
    bill_name = Column(String)
    bill_text = Column(String)
    top_subject = Column(String)

    def __repr__(self):
        return "<US_Bill(bill_num='%s', bill_name='%s', bill_text='%s,', top_subject='%s,')>" % (
            self.bill_num, self.bill_name, self.bill_text, self.top_subject)

In [2]:
class Bill_Subject(Base):
    __tablename__ = 'bill_subject'
    bill_num = Column(String, primary_key=True)
    subject = Column(String, primary_key=True)

    def __repr__(self):
        return "<Bill_Subject(bill_num='%s', subject='%s')>" % (
            self.bill_num, self.subject)

In [ ]:
# Loop through the directory; find data from each json; look in the text directories; open each subdirectory;
# read in each subdirectory json file; find the one with the latest date. Read and store text file associated
# with that directory

In [22]:
import json
import os
import datetime
def get_latest_text_dir(path):
    latest_date = datetime.datetime(1900, 1, 1)
    dir_name = ""
    for status_dir in os.listdir(path):  
        status_file = path + '/' + status_dir + '/data.json'
        with open(status_file) as data_file:
            status_data = json.load(data_file)
            date = datetime.datetime.strptime(status_data['issued_on'], '%Y-%m-%d')
            if (date > latest_date):
                latest_date = date
                dir_name = status_dir
    # print(latest_date)
    return dir_name

In [21]:
# print(get_latest_text_dir('/Users/Joel/Desktop/Insight/data/bills/s/s2039/text-versions'))

2015-09-16 00:00:00
is


In [26]:
import os
from pprint import pprint
bills_dir = '/Users/Joel/Desktop/Insight/data/bills'
j = 1
for bill_type_dir in os.listdir(bills_dir):
    type_dir = bills_dir + '/' + bill_type_dir
    for bill_dir in os.listdir(type_dir):
        bill_path = type_dir + '/' + bill_dir

            #pprint(data)
        text_dir_base = bill_path + '/text-versions'
        if (os.path.isdir(text_dir_base)):
            dir_name = get_latest_text_dir(text_dir_base)
            text_name = text_dir_base + '/' + dir_name + '/document.txt'

            with open(text_name) as text_file:
                text = text_file.readlines()
                if (j == 1):
                    print text
                    j = j + 1
            
            outer_json = bill_path + '/' + 'data.json'
            with open(outer_json) as data_file:    
                bill_data = json.load(data_file)

['\n', '[Congressional Bills 114th Congress]\n', '[From the U.S. Government Printing Office]\n', '[H. Con. Res. 1 Received in Senate (RDS)]\n', '\n', '114th CONGRESS\n', '  1st Session\n', 'H. CON. RES. 1\n', '\n', '\n', '_______________________________________________________________________\n', '\n', '\n', '                   IN THE SENATE OF THE UNITED STATES\n', '\n', '                            January 7, 2015\n', '\n', '                                Received\n', '\n', '_______________________________________________________________________\n', '\n', '                         CONCURRENT RESOLUTION\n', '\n', '\n', ' \n', '     Regarding consent to assemble outside the seat of government.\n', '\n', '    Resolved by the House of Representatives (the Senate concurring), \n', 'That pursuant to clause 4, section 5, article I of the Constitution, \n', 'during the One Hundred Fourteenth Congress the Speaker of the House and \n', 'the Majority Leader of the Senate or their respective de